# MA4702 Laboratorio 3 2022
## Generación de Columnas: Problema sobre Flujo de Costo Mínimo
## Integrantes

- Diego Cespedes
- Gonzalo Ovalle
- Manuel Torres
- Cynthia Vega

## Observaciones

Si tiene alguna observación a la corrección escríbala acá (claridad del enunciado, díficultades con Julia u otros).

# 1. Preparación

Usaremos los paquetes JuMP, Gurobi. Descomente los siguientes comandos si no tiene los paquetes y asegúrese de tener la última versión de JuMP corriendo. Vuelvalos a comentar una vez que los ejecute.

In [32]:
#import Pkg; Pkg.update("JuMP"); Pkg.status("JuMP"); Pkg.add("LightGraphs");

      Status `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
  [4076af6c] JuMP v1.0.0


    Updating registry at `C:\Users\Personal\.julia\registries\General.toml`
   Installed PDMats ─────────── v0.11.8
   Installed Qt5Base_jll ────── v5.15.3+1
   Installed NaNMath ────────── v1.0.0
   Installed StatsAPI ───────── v1.3.0
   Installed DiffRules ──────── v1.11.0
   Installed Preferences ────── v1.3.0
   Installed Parsers ────────── v2.3.1
   Installed StaticArrays ───── v1.4.4
   Installed LogExpFunctions ── v0.3.14
   Installed DataAPI ────────── v1.10.0
   Installed StructArrays ───── v0.6.6
   Installed Latexify ───────── v0.15.15
   Installed ForwardDiff ────── v0.10.27
   Installed MathOptInterface ─ v1.2.0
  No Changes to `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
    Updating `C:\Users\Personal\.julia\environments\v1.7\Manifest.toml`
  [9a962f9c] ↑ DataAPI v1.9.0 ⇒ v1.10.0
  [b552c78f] ↑ DiffRules v1.10.0 ⇒ v1.11.0
  [f6369f11] ↑ ForwardDiff v0.10.25 ⇒ v0.10.27
  [23fbe1c1] ↑ Latexify v0.15.14 ⇒ v0.15.15
  [2ab3a3ac] ↑ LogExpFunctions v0.3.12 ⇒ v0.3.1

In [34]:
using JuMP, Gurobi, LightGraphs
# Abrir un solo ambiente de Gurobi, Jupyter se marea si se crean demasiados modelos durante en una misma celda.
const GUROBI_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09


┌ Info: Precompiling LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1423
┌ Warning: Module StaticArrays with build ID 150854491537301 is missing from the cache.
│ This may mean StaticArrays [90137ffa-7385-5640-81b9-e52037218182] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module StaticArrays with build ID 150854491537301 is missing from the cache.
│ This may mean StaticArrays [90137ffa-7385-5640-81b9-e52037218182] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Info: Skipping precompilation since __precompile__(false). Importing LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d].
└ @ Base loading.jl:1124
┌ Info: Precompiling ArnoldiMethod [ec485272-7323-5ecc-a04f-4719b315124d]
└ @ Base loading.jl:1423
┌ Warning: Module StaticArrays with build ID 150854491537301 is missing from the cache.
│ This may mean StaticArrays [90137ffa-7385-5640-81b9-e520372181

Gurobi.Env(Ptr{Nothing} @0x00000000014138e0, false, 0)

# 2. Funciones Auxiliares

Puede obtener la instancia de prueba para sus programas gracias a la siguiente función:

In [35]:
# Crea la instancia de prueba para el problema de flujo de costo mínimo.
function instancia()
    N=10
    d=10
    c = [25 27 11 25 19 18 14 7 14 3000; 17 14 5 14 30 14 19 27 28 22; 24 11 5 24 24 4 23 23 20 3; 14 17 14 6 1 12 17 22 1 3; 15 30 17 15 9 28 20 20 10 22; 21 9 14 3 18 13 14 1 19 30; 20 18 29 4 18 19 26 5 20 4; 25 14 5 24 5 10 29 21 27 18; 5 17 17 4 17 4 3 17 14 16; 21 11 1 30 17 1 19 14 14 8]
    u = [2 2 3 1 4 1 3 1 4 10; 4 3 2 4 4 1 1 2 3 2; 3 3 1 3 4 2 3 3 3 4; 4 4 3 4 3 3 4 2 4 3; 3 3 2 2 2 4 1 3 4 4; 2 2 4 1 2 4 2 1 2 1; 4 4 1 3 2 1 2 3 1 2; 1 4 1 4 3 4 3 2 1 2; 1 2 2 4 4 4 1 2 3 2; 1 2 3 3 1 3 1 1 2 3]
    s = 1
    t = N
    return N,d,u,c,s,t
end

instancia (generic function with 1 method)

# 3. Problemas

Escriba sus respuestas aquí, tanto los códigos en Julia como respuestas a preguntas en Markdown.

## P1
Puede apoyarse libremente en el siguiente template.

In [71]:
function resolverFlujoCostoMinimo(N,d,u,c,s,t)
    modelo = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "Presolve"=>0, "Cuts"=>0, "Heuristics"=>0, "Threads"=>1))
    set_silent(modelo)
    # Renombrar parametros para usarlos
    cantidad_de_vertices = N
    """Variables"""
    @variable(modelo, f[i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices])
    """Restricciones"""
    for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices
        # El flujo en una arista es acotado superiormente por su capacidad 
        @constraint(modelo, f[i,j] <= u[i,j])
        # El flujo en una arista es no negativo
        @constraint(modelo, f[i,j] >= 0)
    end
    # Restriccion de los cortes fuera del nodo inicial
    for i in 1:cantidad_de_vertices-1
        if i != s
            @constraint(modelo, sum(f[i,j] for j in 1:cantidad_de_vertices) - sum(f[j,i] for j in 1:cantidad_de_vertices) == 0)
        end
    end
    # Restriccion del corte en el nodo inicial
    @constraint(modelo, sum(f[s,j] for j in 1:cantidad_de_vertices) - sum(f[j,s] for j in 1:cantidad_de_vertices) == d)
    """Funcion objetivo y optimizer"""
    @objective(modelo, Min, sum(f[i,j]*c[i,j] for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices))
    optimize!(modelo);
    
    return objective_value(modelo), value.(modelo[:f])
end

resolverFlujoCostoMinimo (generic function with 1 method)

In [72]:
N,d,u,c,s,t = instancia()
@time valor, flujo = resolverFlujoCostoMinimo(N,d,u,c,s,t)
print("El valor es: ", valor)

Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
  0.259522 seconds (271.97 k allocations: 14.801 MiB, 99.35% compilation time)
El valor es: 186.0

## P2
Puede apoyarse libremente en el siguiente template.

In [90]:
function estar(i,j,camino)
        n=length(camino)
        valor=0
        for k in 1:n-1
            if i==camino[k] && j==camino[k+1]
                valor=1 
            end
        end
    return valor
end

estar (generic function with 1 method)

In [100]:
function costocamino(camino)
    suma=0
    for i in 1:length(camino)-1
        suma = suma + c[i,i+1]
    end
    return suma
end    

costocamino (generic function with 1 method)

In [101]:
costocamino([1,2,3])

32

In [120]:
function resolverProblemaMaestro(N,d,u,c,s,t,Caminos)
    maestro = Model(optimizer_with_attributes( () -> Gurobi.Optimizer(GUROBI_ENV), "Presolve"=>0, "Cuts"=>0, "Heuristics"=>0, "Threads"=>1)) 
    set_silent(maestro)

#     @variable
#     @objective
#     @constraints
    cantidad_de_vertices = N
    """Variables"""
    
    @variable(maestro, lambda[P in 1:length(Caminos)])
    """Restricciones"""
    # Restriccion 1
    @constraint(maestro, sum(lambda[i] for i in 1:length(Caminos)) == d)
    # Restriccion 2
    for i in 1:cantidad_de_vertices, j in 1:cantidad_de_vertices
        @constraint(maestro, sum( -estar(i,j,k)*lambda[k] for k in 1:length(Caminos)) >= - u[i,j])
    end
        
    
    # Restriccion 3
    """Funcion objetivo y solver"""
    @objective(maestro, Min, sum( lambda[k]*costocamino(Caminos[k]) for k  in 1:length(Caminos)))
    
    
    # Paso 2. Resolución.
    optimize!(maestro)
    if termination_status(maestro) != MOI.OPTIMAL
        @warn("No hay optimo")
    end
    valor_objetivo = objective_value(maestro)
    λvalues = value.(maestro[:lambda])
    yvalues = dual.(maestro[:capacidad])
    zvalues = dual.(maestro[:demanda])
    return valor_objetivo, λvalues, yvalues, zvalues
end


resolverProblemaMaestro (generic function with 1 method)

## P3
Puede apoyarse libremente en el siguiente template.

In [121]:
function reconstruirFlujo(Caminos,λvalues,N)
    F = zeros(N,N)
    for i in 1:N, j in 1:N
        f(i,j)= sum(lambda[P]*estar(i,j,P) for P in Caminos)
    end
    return F , lambdas
end

reconstruirFlujo (generic function with 2 methods)

## P4
Puede apoyarse libremente en el siguiente template.

In [122]:




function resolverProblemaPricing(N,c,s,t,zvalues,yvalues)
    l=zeros(N,N)
    for i in 1:N,j in 1:N
        l[i,j]=c[i,j]+y[i,j]
    end
    M=zeros(Int64,N,N)
    for i in 1:N
        for j in 1:N
            M[i,j]=1
        end
    end
    G      = DiGraph(M)
    arbol  = dijkstra_shortest_paths(G, s, l)
    corto  = arbol[t]
    optimo = zvalues-corto
    camino=enumerate_paths(arbol,t)

    if optimo <= 0
        return true, []
    else
        #Luego de resolver el pricing, puede serle útil investigae sobre la función enumerate_paths(arbol,t), que devuelve el único s-t camino en la forma que queremos.
        return false, camino
    end
end

resolverProblemaPricing (generic function with 1 method)

## P5
Puede apoyarse libremente en el siguiente template.

In [123]:
function resolverGeneracionColumnas(N,d,u,c,s,t)
    Caminos=Vector{Int}[]
    push!(Caminos, [s,t])
    
    maxcols=300 # damos una cota en el número de columnas a usar.

    while length(Caminos) <= maxcols 
        println(string("Iteración:", length(Caminos)))
        
        # Paso 1. Resolver problema maestro
        valor_objetivo, λvalues, yvalues, zvalues = resolverProblemaMaestro(N,d,u,c,s,t,Caminos)
        
        # Paso 2. Resolver problema pricing
        bool, nuevo_camino = resolverProblemaPricing(N,c,s,t,zvalues,yvalues)
                
        # Paso 3. Revisar si óptimo global o si se necesita un nuevo camino
        if bool
            println("Óptimo")
            return Caminos, λvalues, valor_objetivo
        else
            push!(Caminos, nuevo_camino)
        end
    end
    
    # Se alcanzó el número de iteraciones
    println("Subóptimo")
    return Caminos, λvalues, valor_objetivo
end

resolverGeneracionColumnas (generic function with 1 method)

In [124]:
N,d,u,c,s,t = instancia()
@time Caminos, λvalues, valor_objetivo = resolverGeneracionColumnas(N,d,u,c,s,t)
print("El valor es: ", valor_objetivo)



Iteración:1


LoadError: KeyError: key :λ not found

Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
